## Compute bio-optical ocean phase function

... as described in [Chowdhary et al. 2020] for atmosphere-ocean system IV and compare to the tabulated values provided in their supplemental material for validation.

In [1]:
import os
import numpy as np

import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ipywidgets import widgets, interact

In [2]:
data_dm_mts = np.load('../pf_data/pf_dm_mts.npy')
data_ph_mts = np.load('../pf_data/pf_ph_mts.npy')
data_dm_mish = np.load('../pf_data/pf_dm_mish.npy')
data_ph_mish = np.load('../pf_data/pf_ph_mish.npy')
data_mix_lo_mts = np.load('../pf_data/pf_mix_lo_mts.npy')
data_mix_hi_mts = np.load('../pf_data/pf_mix_hi_mts.npy')
data_mix_lo_mish = np.load('../pf_data/pf_mix_lo_mish.npy')
data_mix_hi_mish = np.load('../pf_data/pf_mix_hi_mish.npy')

# Divide Mishchenko data by 4pi since it does not divide by 4pi to normalize
data_dm_mish[:,1] /= 4. * np.pi
data_ph_mish[:,1] /= 4. * np.pi
data_mix_lo_mish[:,1] /= 4. * np.pi
data_mix_hi_mish[:,1] /= 4. * np.pi

# Load reference data
pf_file_lo = '../pf_data/hydrosol_chl_0_03.csv'
pf_file_hi = '../pf_data/hydrosol_chl_3_0.csv'

data_ch = np.genfromtxt(pf_file_lo, delimiter=',', skip_header=1)

theta_ch = data_ch[:,0]
m11_ch   = data_ch[:,1]
m12_ch   = data_ch[:,2] / m11_ch
m33_ch   = data_ch[:,3] / m11_ch
m34_ch   = data_ch[:,4] / m11_ch

data_mix_lo_ch = np.column_stack((theta_ch, m11_ch, m12_ch, m33_ch, m34_ch))

data_ch = np.genfromtxt(pf_file_hi, delimiter=',', skip_header=1)

theta_ch = data_ch[:,0]
m11_ch   = data_ch[:,1]
m12_ch   = data_ch[:,2] / m11_ch
m33_ch   = data_ch[:,3] / m11_ch
m34_ch   = data_ch[:,4] / m11_ch

data_mix_hi_ch = np.column_stack((theta_ch, m11_ch, m12_ch, m33_ch, m34_ch))

In [3]:
data_select  = widgets.ToggleButtons(options = ['detritus + plankton (low chl)', 'detritus + plankton (high chl)', 'detritus only', 'plankton only'], 
                                     description = '', 
                                     layout = widgets.Layout(width = 'auto'),
                                     style = {"button_width": "auto"},
                                     disabled = False)

data_label = widgets.Label('phase function',
                           layout = widgets.Layout(width = '120px'))

ui = widgets.HBox(children = [data_label, data_select])

# Make 2x2 figure
fig = go.FigureWidget(make_subplots(rows = 2, cols = 2, subplot_titles = ['m11', 'm12', 'm33', 'm34']))

# Fill in figure with line plots
for i in range(2):
    for j in range(2):
        k = i*2 + j
        
        # Reference data
        fig.add_trace(go.Scatter(
            x = np.zeros(180),
            y = np.zeros(180),
            mode = 'lines',
            name = 'Chowdhary',
            legendgroup = 'Chowdhary',
            line = dict(
                        width = 2,
                        color = '#636EFA'
                   ),
            showlegend = (i == 0 and j == 0)
            ),
            row = i+1,
            col = j+1
        )

        # Mishchenko data
        fig.add_trace(go.Scatter(
            x = np.zeros(180),
            y = np.zeros(180),
            mode = 'lines',
            name = 'Mishchenko',
            legendgroup = 'Mishchenko',
            line = dict(
                        width = 2,
                        color = '#00CC96'
                   ),
            showlegend = (i == 0 and j == 0)
            ),
            row = i+1,
            col = j+1
        )
        
        # Mitsuba data
        fig.add_trace(go.Scatter(
            x = np.zeros(180),
            y = np.zeros(180),
            mode = 'lines',
            name = 'Mitsuba',
            legendgroup = 'Mitsuba',
            line = dict(
                        width = 2,
                        color = '#EF553B'
                   ),
            showlegend = (i == 0 and j == 0)
            ),
            row = i+1,
            col = j+1
        )
        
fig.update_layout(
    height = 700,
    title = "Mueller matrix elements",
    xaxis_title = "theta (deg)",
    yaxis_title = "value",
    showlegend = True,
    legend = dict(
        orientation = 'h',
        xanchor = "center",
        yanchor = "top",
        y=1.1,
        x=0.5
    )
)

fig.update_yaxes(
    type = 'log', 
    row = 1,
    col = 1
)

def update(data_select):
    # Load data
    if data_select == 'detritus + plankton (low chl)':
        data_mts = data_mix_lo_mts
        data_mish = data_mix_lo_mish
        data_ch = data_mix_lo_ch
    elif data_select == 'detritus + plankton (high chl)':
        data_mts = data_mix_hi_mts
        data_mish = data_mix_hi_mish
        data_ch = data_mix_hi_ch
    elif data_select == 'detritus only':
        data_mts = data_dm_mts
        data_mish = data_dm_mish
        data_ch = np.zeros_like(data_dm_mts)
    elif data_select == 'plankton only':
        data_mts = data_ph_mts
        data_mish = data_ph_mish
        data_ch = np.zeros_like(data_ph_mts)
    else:
        pass
    
    # Update plots
    with fig.batch_update():
        for i in range(2):
            for j in range(2):
                m = i * 2 + j
                k = m * 3

                fig.data[k    ].x = data_ch[:,0]
                fig.data[k    ].y = data_ch[:,m+1]
                fig.data[k + 1].x = data_mish[:,0]
                fig.data[k + 1].y = data_mish[:,m+1]
                fig.data[k + 2].x = data_mts[:,0]
                fig.data[k + 2].y = data_mts[:,m+1]
        
out = widgets.interactive_output(update, {'data_select': data_select})

display(ui)
fig

FigureWidget({
    'data': [{'legendgroup': 'Chowdhary',
              'line': {'color': '#636EFA', 'width': 2…